In [1]:
# 数据集介绍
# MNIST数据集，100k的训练数据，10k的预测数据，数据由tensorflow中的examples.tutorials.mnist读取 
# 数据集介绍：：Yann LeCun's website
# 由28*28的像素组成输入特征，输出特征为0-9的数字

# 可调节参数：
# --------
# batch_size, initial_weight,二次损失函数,learning_rate,epoch_n
# --------

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data", one_hot = True)

# mini_batch的大小
batch_size = 100
batch_n = mnist.train.num_examples // batch_size

# 创建一个命名空间
with tf.name_scope("input"):
    # # 定义两个placeholder用来feed数据，分别代表x和y --784列和10列(one-hot)
    x = tf.placeholder(tf.float32, [None, 784], name = "x_input")
    y = tf.placeholder(tf.float32, [None, 10], name = "y_input")

# tensorflow中定义一个函数，summary变量的所有特点    
def variables_summary(variable):
    with tf.name_scope("summary"):
        # 通过summary.scalar建立对标量的追踪
        mean = tf.reduce_mean(variable)
        tf.summary.scalar("mean", mean)
        stddev = tf.sqrt(tf.reduce_mean(tf.square(variable - mean)))
        tf.summary.scalar("stddev", stddev)
        tf.summary.histogram("histogram", variable)

# # ----
# # 构建多分类回归
# # 定义weight和bias，初始化分别为正态随机和0.0
with tf.name_scope("lager"):
    with tf.name_scope("weight"):
        initial_weight = tf.random_normal([784, 10])
        weight = tf.Variable(initial_weight)
        variables_summary(weight) # 利用summary函数观察一下weight的变化情况
    with tf.name_scope("bias"):
        bias = tf.Variable(tf.zeros([10]))
        variables_summary(bias) # 利用summary观察一下bias的变化情况
    with tf.name_scope("mat_and_plus"):
        a = tf.matmul(x, weight) + bias
    with tf.name_scope("softmax"):
        y_head = tf.nn.softmax(a)

# # 定义二次损失函数并依据梯度下降法进行训练 -- 这样梯度下降的train就变成了x和y的函数
with tf.name_scope("loss"):
    learning_rate = 0.1
    loss = tf.reduce_mean(tf.square(y - y_head))
    tf.summary.scalar("loss", loss) # 利用summary观察一下loss的变化
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_head, 1)) # tf.argmax找到x中等于1的最大的id
    correction = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) # tf.cast 转换类型，将bool转为float，从而求得准确率
    tf.summary.scalar("accuracy", correction)

# 将summary的scalar merge
merge = tf.summary.merge_all()
    
    
# 迭代500次，进行mini_batch梯度下降
epoch_n = 50
with tf.Session() as session:
    session.run(init)
    writer = tf.summary.FileWriter(logdir = "d://tensorboardLogDir/logs/", graph = session.graph) # 最好不要放到带中文字符的路径
    for step in range(epoch_n):
        for batch in range(batch_n):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, summary = session.run([train, merge], feed_dict= {x: batch_x, y: batch_y})
        writer.add_summary(summary, step) # 每执行一个周期将summary中的信息写入log、
        corr = session.run(correction, feed_dict= {x: mnist.test.images, y: mnist.test.labels}) # 基于测试集对准确率进行测试
        print("in iteration " + str(step) + "the accuracy is : " + str(corr)) # 打印准确率
# 这里看似有问题，其实没问题，因为图没变，DAG对输入的batch依次执行梯度下降法，
# 并执行epoch_n个周期，权重会更新epoch_n * batch_n次

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
in iteration 0the accuracy is : 0.1015
in iteration 1the accuracy is : 0.1158
in iteration 2the accuracy is : 0.1278
in iteration 3the accuracy is : 0.1402
in iteration 4the accuracy is : 0.1511
in iteration 5the accuracy is : 0.1591
in iteration 6the accuracy is : 0.167
in iteration 7the accuracy is : 0.175
in iteration 8the accuracy is : 0.1833
in iteration 9the accuracy is : 0.1906
in iteration 10the accuracy is : 0.198
in iteration 11the accuracy is : 0.2048
in iteration 12the accuracy is : 0.2097
in iteration 13the accuracy is : 0.2148
in iteration 14the accuracy is : 0.2216
in iteration 15the accuracy is : 0.2278
in iteration 16the accuracy is : 0.2334
in iteration 17the accuracy is : 0.2392
in iteration 18the accuracy is : 0.2456
in iteration 19the accuracy is : 0.2518
in iteration 20the 